In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_type = "azure"
openai.api_base = "https://husein-ai9-aiservices1611641965.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''
engine = 'gpt-35-turbo-16k'

In [3]:
def predict(messages):
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=2048,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [4]:
system = 'Above is a conversation between a user and an intelligent assistant. You suppose to simulate conversation between the user and the assistant. Bear in mind your major request is to ask the assistant to generate some material. So you can ask the assistant either to make it more detailed, add more related information, or any other request to improve the generated material. Be creative and diverse in your request. Make the response short and the language casual.'
system_malay = 'Above is a conversation between a user and an intelligent assistant. You suppose to simulate conversation between the user and the assistant in malay language. Bear in mind your major request is to ask the assistant to generate some material. So you can ask the assistant either to make it more detailed, add more related information, or any other request to improve the generated material. Be creative and diverse in your request. Make the response short and the language casual.'
next_step = 'Above is a conversation between a user and an intelligent assistant. Now suppose you are the user, generate response according to the generated material to continue the conversation.'

In [5]:
!mkdir ultrachat-glaive_coder_raw_text
# !rm ultrachat-textbook/*.json

mkdir: cannot create directory ‘ultrachat-glaive_coder_raw_text’: File exists


In [6]:
def ultrachat(row, n = 1):
    
    results = [
        {'role': 'user', 'content': row['instruction']},
        {'role': 'assistant', 'content': row['output']},
    ]
    
    initial = row['instruction'].strip()
    messages = [
        {'role': 'system', 'content': system_malay},
        {'role': 'user', 'content': initial + ', jawab dalam bahasa melayu'},
        {'role': 'assistant', 'content': row['output']},
    ]
    
    for _ in range(n):
    
        messages_temp = messages + [
            {'role': 'user', 'content': 'Now suppose you are the user and you always doubt with the answer and eager to know better, so say something in malay to continue the conversation based on given context.'}
        ]
        r2 = predict(messages_temp)
        if r2 is None:
            break

        results.append({
            'role': 'user', 'content': r2,
        })
        messages.append({
            'role': 'user', 'content': r2 + ', jawab dalam bahasa melayu',
        })

        r = predict(messages)
        if r is None:
            break
        results.append({
            'role': 'assistant', 'content': r,
        })
        messages.append({
            'role': 'assistant', 'content': r,
        })
    
    return results

In [7]:
from glob import glob

files = sorted(glob('/home/husein/ssd3/instructions/glaive_coder_raw_text/*.json'), key = lambda x: int(x.split('/')[-1].replace('.json', '')))
len(files)

119122

In [8]:
rejected_instructions = [
    'tidak dapat melakukan penulisan semula', 
    'maaf', 
    'not able to fulfill your request',
    'saya tidak dapat membantu',
    'dengan permintaan itu',
    'feel free to ask',
    'saya tidak bisa',
]

In [14]:
def generate(f):
    index = int(os.path.split(f)[1].replace('.json', ''))
    filename = f'ultrachat-glaive_coder_raw_text/{index}.json'
    if os.path.exists(filename):
        return
    
    try:
        with open(f) as fopen:
            data = json.load(fopen)
    except:
        return
        
    if any([r in data['instruction'].lower() for r in rejected_instructions]):
        return
    
    r = ultrachat(data, n = random.randint(1, 4))
    with open(filename, 'w') as fopen:
        json.dump(r, fopen)

In [10]:
generate(files[4])

In [11]:
# !ls ultrachat-glaive_coder_raw_text

In [12]:
with open('ultrachat-glaive_coder_raw_text/4.json') as fopen:
    data = json.load(fopen)
    
data

[{'role': 'user',
  'content': "Saya sedang belajar Python dan perasan penggunaan yang meluas pada kewataan 'self' dalam definisi kelas. Datang dari latar belakang Java, saya sering lupa untuk memasukkan 'self' ketika merujuk kepada pembolehubah ahli. Sebagai contoh, daripada menulis 'self.rect.centerx', saya cenderung menulis 'rect.centerx' kerana saya melihat 'rect' sudah menjadi pembolehubah ahli kelas. Adakah ada cara untuk mengelakkan keperluan untuk mengekori semua pembolehubah ahli dengan 'self' dalam Python?"},
 {'role': 'assistant',
  'content': "Di dalam Python, keperluan untuk menggunakan 'self' untuk merujuk kepada pembolehubah ahli dalam definisi kelas adalah satu ciri yang asas. Walau bagaimanapun, terdapat satu cara untuk mengelakkan keperluan tersebut dalam beberapa keadaan dengan menggunakan pembolehubah yang didefinisikan dalam fungsi kelas, iaitu dengan disertakan dengan tanda garis bawah dua kali (dunder) ('__') di hadapan nama pembolehubah.\n\nContohnya, anda boleh

In [15]:
from tqdm import tqdm

max_worker = 3
for i in tqdm(range((len(files) // 6) * 0, (len(files) // 6) * 1, max_worker)):
    b = files[i: i + max_worker]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f): f for f in b}

        for future in as_completed(futures):
            future.result()

  6%|██▎                                | 428/6618 [2:32:41<57:57:29, 33.71s/it]

'content'


 12%|████                               | 779/6618 [5:55:45<53:38:04, 33.07s/it]

'content'


 12%|████▎                              | 812/6618 [6:12:42<57:10:22, 35.45s/it]

...

 94%|████████████████████████████████  | 6243/6618 [44:52:28<2:27:01, 23.52s/it]

'content'


 95%|████████████████████████████████▍ | 6314/6618 [45:18:48<2:16:14, 26.89s/it]

'content'


100%|████████████████████████████████████| 6618/6618 [47:15:29<00:00, 25.71s/it]
